#Importent libraries


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import os
print(os.listdir("/content/Captcha"))



['wyc25.png', 'w2n7e.png', '76n7p.png', 'bw5ym.png', '58b5m.png', '3dgmf.png', 'dmxp8.png', 'ywn6f.png', 'gegw4.png', 'pcpg6.png', 'y5dpp.png', 'pm47f.png', '3bnfnd.png', 'nfcwy.png', 'ncww7.png', 'n2by7.png', '33f7m.png', 'm3b5p.png', 'yw667.png', '5325m.png', '8ne4g.png', 'xbem6.png', '5dxnm.png', 'pe4xn.png', 'edwny.png', 'c6f8g.png', 'm457d.png', 'pp546.png', '43gey.png', '33b22.png', '6cm6m.png', 'nfd8g.png', 'nm248.png', 'yx2d4.png', '325fb.png', 'fncnb.png', '7gce6.png', 'x8e8n.png', 'cd6p4.png', '7xcyd.png', 'n6f4b.png', '823p2.png', 'gd4mf.png', 'f22bn.png', 'mmg38.png', '4cfw8.png', '8e32m.png', 'c753e.png', 'nnn57.png', '87nym.png', 'c43b4.png', 'm8m4x.png', '4gycb.png', '5bg8f.png', 'n5x2n.png', '6p2ge.png', 'wecfd.png', 'g888x.png', 'excmn.png', '5n3w4.png', 'enn7n.png', 'bnc5f.png', '8y6b3.png', 'c2pg6.png', 'cc845.png', 'n3bm6.png', '6fgdw.png', 'nwg2m.png', '3nnpw.png', '38n57.png', 'fyfbn.png', '785n4.png', 'n7g4f.png', '23n88.png', '5nnff.png', 'd4ppy.png', 'w4cdc.png

In [ ]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np

#Init main values
symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
num_symbols = len(symbols)
img_shape = (50, 200, 1)

#Preprocessing


In [ ]:
def preprocess_data():
    n_samples = len(os.listdir('/content/Captcha'))
    X = np.zeros((n_samples, 50, 200, 1)) #1070*50*200
    y = np.zeros((5, n_samples, num_symbols)) #5*1070*36

    for i, pic in enumerate(os.listdir('/content/Captcha')):
        # Read image as grayscale
        img = cv2.imread(os.path.join('/content/Captcha', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 6:
            # Scale and reshape image
            img = img / 255.0
            img = np.reshape(img, (50, 200, 1))
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs

    # Return final data
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]

In [ ]:
X_test

array([[[[0.75294118],
         [0.75294118],
         [0.75294118],
         ...,
         [0.98431373],
         [0.98431373],
         [0.98431373]],

        [[0.75294118],
         [0.75294118],
         [0.75294118],
         ...,
         [0.98431373],
         [0.98431373],
         [0.98431373]],

        [[0.75294118],
         [0.75294118],
         [0.75294118],
         ...,
         [0.98431373],
         [0.98431373],
         [0.98431373]],

        ...,

        [[0.76470588],
         [0.76470588],
         [0.76470588],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        [[0.76470588],
         [0.76470588],
         [0.76470588],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]],

        [[0.76470588],
         [0.76470588],
         [0.76470588],
         ...,
         [0.99607843],
         [0.99607843],
         [0.99607843]]],


       [[[0.75294118],
         [0.75294118],
         [0.75

In [ ]:
X_train.shape

(970, 50, 200, 1)

In [ ]:
y_train.shape

(5, 970, 36)

In [ ]:
y_test.shape

(5, 100, 36)

#Creating model


In [ ]:
def create_model():
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7

    # Get flattened vector and make 5 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3)
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)

    # Compile model and return it
    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [ ]:
model=create_model();
model.summary();

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50, 200, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 50, 200, 16)  160         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 25, 100, 16)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 25, 100, 32)  4640        ['max_pooling2d[0][0]']      

#Fit model on train data set

In [ ]:
#model = create_model()
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=30,verbose=1, validation_split=0.2)



Epoch 1/30
25/25 [==============================] - 10s 44ms/step - loss: 17.6831 - dense_1_loss: 3.5635 - dense_3_loss: 3.5667 - dense_5_loss: 3.5549 - dense_7_loss: 3.5017 - dense_9_loss: 3.4963 - dense_1_accuracy: 0.0593 - dense_3_accuracy: 0.0503 - dense_5_accuracy: 0.0567 - dense_7_accuracy: 0.0503 - dense_9_accuracy: 0.0722 - val_loss: 17.5861 - val_dense_1_loss: 3.5358 - val_dense_3_loss: 3.5309 - val_dense_5_loss: 3.4975 - val_dense_7_loss: 3.5000 - val_dense_9_loss: 3.5219 - val_dense_1_accuracy: 0.0928 - val_dense_3_accuracy: 0.0464 - val_dense_5_accuracy: 0.0722 - val_dense_7_accuracy: 0.0670 - val_dense_9_accuracy: 0.1237
Epoch 2/30
25/25 [==============================] - 0s 14ms/step - loss: 16.3234 - dense_1_loss: 3.2796 - dense_3_loss: 3.2958 - dense_5_loss: 3.2801 - dense_7_loss: 3.2360 - dense_9_loss: 3.2318 - dense_1_accuracy: 0.0838 - dense_3_accuracy: 0.0941 - dense_5_accuracy: 0.0902 - dense_7_accuracy: 0.0902 - dense_9_accuracy: 0.0954 - val_loss: 17.6221 - val_d

#Test Model

In [ ]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt#, sum(probs) / 5

In [ ]:
score= model.evaluate(X_test,[y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]],verbose=1)
print('Test Loss and accuracy:', score)

4/4 [==============================] - 0s 33ms/step - loss: 1.8727 - dense_1_loss: 0.1685 - dense_3_loss: 0.2245 - dense_5_loss: 0.4394 - dense_7_loss: 0.4978 - dense_9_loss: 0.5425 - dense_1_accuracy: 0.9500 - dense_3_accuracy: 0.9400 - dense_5_accuracy: 0.9000 - dense_7_accuracy: 0.8400 - dense_9_accuracy: 0.7900
Test Loss and accuracy: [1.8726959228515625, 0.1684735119342804, 0.2245349884033203, 0.4393858313560486, 0.49782150983810425, 0.5424802303314209, 0.949999988079071, 0.9399999976158142, 0.8999999761581421, 0.8399999737739563, 0.7900000214576721]


In [ ]:
# Check model on some samples
model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print("Predict for 8n5p3:")
print(predict('/content/Captcha/8n5p3.png'))

print("Predict for f2m8n:")
print(predict('/content/Captcha/f2m8n.png'))

print("Predict for dce8y:")
print(predict('/content/Captcha/dce8y.png'))

print("Predict for 3eny7:")
print(predict('/content/Captcha/3eny7.png'))


4/4 [==============================] - 0s 8ms/step - loss: 1.8727 - dense_1_loss: 0.1685 - dense_3_loss: 0.2245 - dense_5_loss: 0.4394 - dense_7_loss: 0.4978 - dense_9_loss: 0.5425 - dense_1_accuracy: 0.9500 - dense_3_accuracy: 0.9400 - dense_5_accuracy: 0.9000 - dense_7_accuracy: 0.8400 - dense_9_accuracy: 0.7900
Predict for 8n5p3:
1/1 [==============================] - 0s 216ms/step
8n5p3
Predict for f2m8n:
1/1 [==============================] - 0s 17ms/step
f2n8n
Predict for dce8y:
1/1 [==============================] - 0s 18ms/step
dce8y
Predict for 3eny7:
1/1 [==============================] - 0s 17ms/step
3eny7
